In [12]:
# !python -m pip install mlflow==1.24.1.dev0 --find-links=https://mlflow-snapshots.s3-us-west-2.amazonaws.com/mlflow-1.24.1.dev0-0.0825716d-py2.py3-none-any.whl
# !python -m pip install conda

     |████████████████████████████████| 299 kB 11.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 66 kB 4.9 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for conda: filename=conda-4.3.16-py3-none-any.whl size=336938 sha256=f0b5769781114950e25e2f1190e88ab345b6d038f29bf19d3559f53b1b04fd91
  Stored in directory: /home/akinwilson/.cache/pip/wheels/6e/c5/70/e50a1a99d26cbe940549549b7601d484e0f0bc4b8f0ec49e36
  Created wheel for pycosat: filename=pycosat-0.6.3-cp38-cp38-linux_x86_64.whl size=190930 sha256=ca8334534f0681eff9a6b7b9dd688b01be7f728fedf85e239de402d1b6700791
  Stored in directory: /home/akinwilson/.cache/pip/wheels/71/5b/2d/0a9247760f0f008abc0eae7c0127a8c1cddfe1145e4b170fc0
Successfully built conda pycosat
You should consider upgrading via the '/home/akinwilson/.pyenv/versions/3.8.2/bin/python -m pip install --upgrade pip' command.


In [11]:

import pandas as pd 
import logging
import pickle 

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def get_data():
    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        df = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )
    return df 


df = get_data()
df.drop_column("quality")

AttributeError: 'DataFrame' object has no attribute 'drop_column'

In [24]:
import mlflow 
import mlflow.sklearn 

# mlflow.sklearn.log_model(sk_model,
#                         artifact_path,
#                         conda_env, 
#                         code_paths, 
#                         serialization_format,
#                         registered_model_name,
#                         signature: ModelSignature,
#                         input_example, 
#                         pip_requirements)


from mlflow.models.signature import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet 

# train = df.drop_column("target_label")
# predictions = ... # compute model predictions
# signature = infer_signature(train, predictions)



train, test = train_test_split(df)

# features 
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)

# targets 
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha=0.5
l1_ratio=0.5
lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)
y_hat = lr.predict(test_x)


def get_model_signature(x, y_hat):
    '''inputs are expected to be a dataframe and the prediction of the model'''
    return infer_signature(x, y_hat)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
945,10.2,0.44,0.42,2.0,0.071,7.0,20.0,0.99566,3.14,0.79,11.1


In [14]:
import mlflow
from dotenv import load_dotenv
from pprint import pprint
from mlflow.tracking import MlflowClient
client = MlflowClient()

In [18]:
# [x for x in dir(client) if not x.startswith("_")]

['create_experiment',
 'create_model_version',
 'create_registered_model',
 'create_run',
 'delete_experiment',
 'delete_model_version',
 'delete_model_version_tag',
 'delete_registered_model',
 'delete_registered_model_tag',
 'delete_run',
 'delete_tag',
 'download_artifacts',
 'get_experiment',
 'get_experiment_by_name',
 'get_latest_versions',
 'get_metric_history',
 'get_model_version',
 'get_model_version_download_uri',
 'get_model_version_stages',
 'get_registered_model',
 'get_run',
 'list_artifacts',
 'list_experiments',
 'list_registered_models',
 'list_run_infos',
 'log_artifact',
 'log_artifacts',
 'log_batch',
 'log_dict',
 'log_figure',
 'log_image',
 'log_metric',
 'log_param',
 'log_text',
 'rename_experiment',
 'rename_registered_model',
 'restore_experiment',
 'restore_run',
 'search_model_versions',
 'search_registered_models',
 'search_runs',
 'set_experiment_tag',
 'set_model_version_tag',
 'set_registered_model_tag',
 'set_tag',
 'set_terminated',
 'transition_mode

In [2]:
import os
print("Tracking server uri before loading env vars:")
print(mlflow.tracking.get_tracking_uri())
print("\n")
load_dotenv()
print("Tracking server uri after loading env vars:")
print(mlflow.tracking.get_tracking_uri())
DEFAULT_ARTIFACTS_URI = os.environ['MLFLOW_TRACKING_URI']

Tracking server uri before loading env vars:
file:///home/akinwilson/Projects/mlops/app/mlruns


Tracking server uri after loading env vars:
http://mlops-alb-development-2008207679.eu-west-2.elb.amazonaws.com/mlops/


In [3]:
from mlflow.store.artifact.mlflow_artifacts_repo import MlflowArtifactsRepository
print(f"Current artifacts associated to {DEFAULT_ARTIFACTS_URI}:")
MlflowArtifactsRepository(DEFAULT_ARTIFACTS_URI).list_artifacts()

Current artifacts associated to http://mlops-alb-development-2008207679.eu-west-2.elb.amazonaws.com/mlops/:


[]

### Defining sampling model training routine with mlflow logging

- parameter tracking
- metric tracking
- model logging 

In [4]:
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
# from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2



def get_model_signature(x, y_hat):
    '''inputs are expected to be a dataframe and the prediction of the model'''
    return infer_signature(x, y_hat)


def training():
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha =  0.5
    l1_ratio = 0.5

    exp_id = mlflow.create_experiment("testing-V5")
    print(mlflow.get_experiment(exp_id))
    
    mlflow.start_run(experiment_id=exp_id)
    
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
    print(f"Elasticnet model (alpha={alpha}, l1_ratio={l1_ratio}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    # param logging
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    # metric logging
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    # model logging
    mlflow.sklearn.log_model(sk_model=lr,
                            artifact_path="model",
                            registered_model_name="ElasticnetWineModel",
                            signature=get_model_signature(test_x, y_hat),
                            input_example=test_x.head(1))
    mlflow.end_run()

In [5]:
training()

<Experiment: artifact_location='mlflow-artifacts:/1', experiment_id='1', lifecycle_stage='active', name='testing-V5', tags={}>
Elasticnet model (alpha=0.5, l1_ratio=0.5):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


Successfully registered model 'ElasticnetWineModel'.
2022/04/18 11:38:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetWineModel, version 1
Created version '1' of model 'ElasticnetWineModel'.


In [ ]:
mlflow.end_run()

# Add or updating model description

In [15]:
client = MlflowClient()

desc =""" Model trained on  two datasets are related to red and white variants of the Portuguese 
        'Vinho Verde' wine. For more details, consult the reference [Cortez et al., 2009]."""


client.update_model_version(
    name="ElasticnetWineModel",
    version=1,
    description=desc
)


<ModelVersion: creation_timestamp=1650278328899, current_stage='Production', description=(' Model trained on  two datasets are related to red and white variants of the '
 'Portuguese \n'
 "        'Vinho Verde' wine. For more details, consult the reference [Cortez "
 'et al., 2009].'), last_updated_timestamp=1650279129615, name='ElasticnetWineModel', run_id='6c70852082c643e080835b6405e014ed', run_link='', source='mlflow-artifacts:/1/6c70852082c643e080835b6405e014ed/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

# Show registered Models

In [8]:
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)

{   'creation_timestamp': 1650278328791,
    'description': '',
    'last_updated_timestamp': 1650278328899,
    'latest_versions': [   <ModelVersion: creation_timestamp=1650278328899, current_stage='None', description='', last_updated_timestamp=1650278328899, name='ElasticnetWineModel', run_id='6c70852082c643e080835b6405e014ed', run_link='', source='mlflow-artifacts:/1/6c70852082c643e080835b6405e014ed/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'ElasticnetWineModel',
    'tags': {}}


In [9]:
dict(rm)

{'creation_timestamp': 1650278328791,
 'description': '',
 'last_updated_timestamp': 1650278328899,
 'latest_versions': [<ModelVersion: creation_timestamp=1650278328899, current_stage='None', description='', last_updated_timestamp=1650278328899, name='ElasticnetWineModel', run_id='6c70852082c643e080835b6405e014ed', run_link='', source='mlflow-artifacts:/1/6c70852082c643e080835b6405e014ed/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>],
 'name': 'ElasticnetWineModel',
 'tags': {}}

# Transition model to stage

In [10]:
client.transition_model_version_stage(
    name="ElasticnetWineModel",
    version=1,
    stage="Production"
)

<ModelVersion: creation_timestamp=1650278328899, current_stage='Production', description='', last_updated_timestamp=1650278613192, name='ElasticnetWineModel', run_id='6c70852082c643e080835b6405e014ed', run_link='', source='mlflow-artifacts:/1/6c70852082c643e080835b6405e014ed/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

# Serving model
_to do: need to include mlflowProject to ensure dependencies are all present_

In [13]:
# Set environment variable for the tracking URL where the Model Registry resides
!export MLFLOW_TRACKING_URI=DEFAULT_ARTIFACTS_URI
# Serve the production model from the model registry
!mlflow models serve -m "models:/ElasticnetWineModel/Production"

2022/04/18 11:46:33 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
Traceback (most recent call last):
  File "/home/akinwilson/.pyenv/versions/3.8.2/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/home/akinwilson/.local/lib/python3.8/site-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/home/akinwilson/.local/lib/python3.8/site-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/home/akinwilson/.local/lib/python3.8/site-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/akinwilson/.local/lib/python3.8/site-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/akinwilson/.local/lib/python3.8/site-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/akinwilson/.local/lib/python3.8/site-pack